In [20]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import db_dtypes
import snowflake.connector
import math
import boto3
from datetime import datetime

# pip install google-cloud-bigquery google-auth pandas db-dtypes snowflake-connector-python boto3

###################################
#### NOTES ########################
#####################################
# NEED TO ADD IN LOGI TO MAKE A TALBE THAT HAS THE RUNTIMES OF EACH TABLE THEN THTE LAST RUN TIME WHERE YOU CHOOSE MAX FROM THE PARTICUAL TABLE THEN SAY GREATER THAN 
#### STORE THIS IN SNOWFLAKE WITH TIMESTAMPS 
#### FILTER IT IN BIG QUERY 
# CLEAR OUT THE LOCAL FILES 

snowflake_table = ''

AWS_KEY_ID = ''
AWS_SECRET_KEY = ''

bucket = ''
folder = 'users'
stage_table = 'G'
stage_table_sn = ''


#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "surremor-ocf-69d6a4754b0a.json"

# ACCESS INFORMATION
###########################################
access_key_id =''
secret_access_key=''


# Set up AWS S3 client
s3 = boto3.client('s3',
                  aws_access_key_id='',
                  aws_secret_access_key='',
                  region_name='us-east-1')

# Snowflake connection details
conn = snowflake.connector.connect(
    user='',
    password='!',
    account='',  # Correct format for the account parameter
    warehouse='',
    database='',
    schema='',
    role=''
)
cur = conn.cursor()

lastrun = """
select MAX(TIMESTAMP) 
from GOOGLE_BIGQUERY.EVENT_VIEWS.RUNTIME_HST 
WHERE TABLE_NAME ILIKE '%GOOGLE_BIGQUERY.USERS_VIEWS.USERS_FRDV%';
"""
#LASTRUN = '2024-09-22'

cur.execute(lastrun)


key_path = "bigquerykey.json"
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)


lastrun_result = cur.fetchone()  

if lastrun_result and lastrun_result[0]:
    last_run_time = lastrun_result[0].strftime('%Y-%m-%d')
else:
    last_run_time = None  # Handle case if there is no last run time

query = f"""
SELECT
  *
FROM
  `big query data`
"""
df = client.query(query).to_dataframe()
user_info_normalized = pd.json_normalize(df['user_info'])
geo_normalized = pd.json_normalize(df['geo'])
device_normalized = pd.json_normalize(df['device'])
predictions_normalized = pd.json_normalize(df['predictions'])
privacy_info_normalized = pd.json_normalize(df['privacy_info'])
user_ltv_normalized = pd.json_normalize(df['user_ltv'])

# Join the normalized DataFrames back to the original DataFrame
# Add suffixes to columns to avoid overlap
df_final = df.join([
    user_info_normalized.add_suffix('_user_info'), 
    geo_normalized.add_suffix('_geo'), 
    device_normalized.add_suffix('_device'), 
    privacy_info_normalized.add_suffix('_privacy_info'),
    predictions_normalized.add_suffix('_predictions'),
    user_ltv_normalized.add_suffix('_user_ltv')

])

df_exploded = df_final.explode('audiences')
df_exploded['name'] = df_exploded['audiences'].apply(lambda x: x['name'] if isinstance(x, dict) else None)
df_exploded['membership_start_timestamp_micros'] = df_exploded['audiences'].apply(lambda x: x['membership_start_timestamp_micros'] if isinstance(x, dict) else None)
df_exploded['membership_expiry_timestamp_micros'] = df_exploded['audiences'].apply(lambda x: x.get('membership_expiry_timestamp_micros') if isinstance(x, dict) else None)
df_exploded['npa'] = df_exploded['audiences'].apply(lambda x: x.get('npa') if isinstance(x, dict) else None)
df_exploded['id'] = df_exploded['audiences'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)


# Drop the original nested columns if necessary
df_final = df_final.drop(columns=['user_info', 
                                  'device', 
                                  'geo', 
                                  'predictions', 
                                  'privacy_info',
                                  'user_ltv',
                                  'audiences',
                                  'user_properties'])
df_final.columns = df_final.columns.str.replace('.', '_', regex=False)


numeric_columns = df_final.select_dtypes(include=['float64', 'int64']).columns
df_final[numeric_columns] = df_final[numeric_columns].fillna(0)
string_columns = df_final.select_dtypes(include=['object']).columns
df_final[string_columns] = df_final[string_columns].fillna('Unknown')
boolean_columns = df_final.select_dtypes(include=['bool']).columns
df_final[boolean_columns] = df_final[boolean_columns].fillna(False)
df_final.dropna(axis=1, how='all', inplace=True)

df_final['pseudo_user_id'] = df_final['pseudo_user_id'].astype(object)
df_final['stream_id'] = df_final['stream_id'].astype(object)
df_final['occurrence_date'] = df_final['occurrence_date'].astype(object)
df_final['last_updated_date'] = df_final['last_updated_date'].astype(object)
df_final['last_active_timestamp_micros_user_info'] = df_final['last_active_timestamp_micros_user_info'].astype(object)
df_final['user_first_touch_timestamp_micros_user_info'] = df_final['user_first_touch_timestamp_micros_user_info'].astype(object)
df_final['first_purchase_date_user_info'] = df_final['first_purchase_date_user_info'].astype(object)
df_final['city_geo'] = df_final['city_geo'].astype(object)
df_final['country_geo'] = df_final['country_geo'].astype(object)
df_final['continent_geo'] = df_final['continent_geo'].astype(object)
df_final['region_geo'] = df_final['region_geo'].astype(object)
df_final['operating_system_device'] = df_final['operating_system_device'].astype(object)
df_final['category_device'] = df_final['category_device'].astype(object)
df_final['mobile_brand_name_device'] = df_final['mobile_brand_name_device'].astype(object)
df_final['mobile_model_name_device'] = df_final['mobile_model_name_device'].astype(object)
df_final['unified_screen_name_device'] = df_final['unified_screen_name_device'].astype(object)
df_final['is_limited_ad_tracking_privacy_info'] = df_final['is_limited_ad_tracking_privacy_info'].astype(object)
df_final['is_ads_personalization_allowed_privacy_info'] = df_final['is_ads_personalization_allowed_privacy_info'].astype(object)
df_final['in_app_purchase_score_7d_predictions'] = df_final['in_app_purchase_score_7d_predictions'].astype(object)
df_final['purchase_score_7d_predictions'] = df_final['purchase_score_7d_predictions'].astype(object)
df_final['churn_score_7d_predictions'] = df_final['churn_score_7d_predictions'].astype(object)
df_final['revenue_28d_in_usd_predictions'] = df_final['revenue_28d_in_usd_predictions'].astype(object)
df_final['revenue_in_usd_user_ltv'] = df_final['revenue_in_usd_user_ltv'].astype(object)
df_final['sessions_user_ltv'] = df_final['sessions_user_ltv'].astype(object)
df_final['engagement_time_millis_user_ltv'] = df_final['engagement_time_millis_user_ltv'].astype(object)
df_final['purchases_user_ltv'] = df_final['purchases_user_ltv'].astype(object)
df_final['engaged_sessions_user_ltv'] = df_final['engaged_sessions_user_ltv'].astype(object)
df_final['session_duration_micros_user_ltv'] = df_final['session_duration_micros_user_ltv'].astype(object)

# Define batch size
batch_size = 100000
num_batches = len(df_final) // batch_size + 1

# Loop through and upload batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df_final))
    
    # Slice DataFrame into a batch
    batch_data = df_final[start_idx:end_idx]
    
    # Save batch to a temporary CSV file
    file_name = f'batch_{i+1}.csv'
    batch_data.to_csv(file_name, index=False)
    
    # Upload CSV to S3
    s3.upload_file(file_name,bucket, f'{folder}/' + file_name)

    print(f"Uploaded {file_name} to S3.")

# Load data from S3 into Snowflake table
copy_command = f"""
COPY INTO {stage_table_sn}
FROM 's3://{bucket}/{folder}/{file_name}'
CREDENTIALS = (AWS_KEY_ID = '{AWS_KEY_ID}' AWS_SECRET_KEY = '{AWS_SECRET_KEY}')
FILE_FORMAT = (
    TYPE = 'CSV' 
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    SKIP_HEADER = 1
    ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE
);
"""

cur.execute(copy_command)

merge = """MERGE INTO GOOGLE_BIGQUERY.USERS_VIEWS.USERS_FRDV AS tgt
USING GOOGLE_BIGQUERY.USERS_VIEWS.USERS_STAGE AS src
ON tgt.pseudo_user_id = src.pseudo_user_id
   AND tgt.stream_id = src.stream_id
   AND tgt.occurrence_date = src.occurrence_date
WHEN MATCHED THEN
    UPDATE SET
        tgt.last_updated_date = src.last_updated_date,
        tgt.last_active_timestamp_micros_user_info = src.last_active_timestamp_micros_user_info,
        tgt.user_first_touch_timestamp_micros_user_info = src.user_first_touch_timestamp_micros_user_info,
        tgt.first_purchase_date_user_info = src.first_purchase_date_user_info,
        tgt.city_geo = src.city_geo,
        tgt.country_geo = src.country_geo,
        tgt.continent_geo = src.continent_geo,
        tgt.region_geo = src.region_geo,
        tgt.operating_system_device = src.operating_system_device,
        tgt.category_device = src.category_device,
        tgt.mobile_brand_name_device = src.mobile_brand_name_device,
        tgt.mobile_model_name_device = src.mobile_model_name_device,
        tgt.unified_screen_name_device = src.unified_screen_name_device,
        tgt.is_limited_ad_tracking_privacy_info = src.is_limited_ad_tracking_privacy_info,
        tgt.is_ads_personalization_allowed_privacy_info = src.is_ads_personalization_allowed_privacy_info,
        tgt.in_app_purchase_score_7d_predictions = src.in_app_purchase_score_7d_predictions,
        tgt.purchase_score_7d_predictions = src.purchase_score_7d_predictions,
        tgt.churn_score_7d_predictions = src.churn_score_7d_predictions,
        tgt.revenue_28d_in_usd_predictions = src.revenue_28d_in_usd_predictions,
        tgt.revenue_in_usd_user_ltv = src.revenue_in_usd_user_ltv,
        tgt.sessions_user_ltv = src.sessions_user_ltv,
        tgt.engagement_time_millis_user_ltv = src.engagement_time_millis_user_ltv,
        tgt.purchases_user_ltv = src.purchases_user_ltv,
        tgt.engaged_sessions_user_ltv = src.engaged_sessions_user_ltv,
        tgt.session_duration_micros_user_ltv = src.session_duration_micros_user_ltv
WHEN NOT MATCHED THEN
    INSERT (
        pseudo_user_id,
        stream_id,
        occurrence_date,
        last_updated_date,
        last_active_timestamp_micros_user_info,
        user_first_touch_timestamp_micros_user_info,
        first_purchase_date_user_info,
        city_geo,
        country_geo,
        continent_geo,
        region_geo,
        operating_system_device,
        category_device,
        mobile_brand_name_device,
        mobile_model_name_device,
        unified_screen_name_device,
        is_limited_ad_tracking_privacy_info,
        is_ads_personalization_allowed_privacy_info,
        in_app_purchase_score_7d_predictions,
        purchase_score_7d_predictions,
        churn_score_7d_predictions,
        revenue_28d_in_usd_predictions,
        revenue_in_usd_user_ltv,
        sessions_user_ltv,
        engagement_time_millis_user_ltv,
        purchases_user_ltv,
        engaged_sessions_user_ltv,
        session_duration_micros_user_ltv
    )
    VALUES (
        src.pseudo_user_id,
        src.stream_id,
        src.occurrence_date,
        src.last_updated_date,
        src.last_active_timestamp_micros_user_info,
        src.user_first_touch_timestamp_micros_user_info,
        src.first_purchase_date_user_info,
        src.city_geo,
        src.country_geo,
        src.continent_geo,
        src.region_geo,
        src.operating_system_device,
        src.category_device,
        src.mobile_brand_name_device,
        src.mobile_model_name_device,
        src.unified_screen_name_device,
        src.is_limited_ad_tracking_privacy_info,
        src.is_ads_personalization_allowed_privacy_info,
        src.in_app_purchase_score_7d_predictions,
        src.purchase_score_7d_predictions,
        src.churn_score_7d_predictions,
        src.revenue_28d_in_usd_predictions,
        src.revenue_in_usd_user_ltv,
        src.sessions_user_ltv,
        src.engagement_time_millis_user_ltv,
        src.purchases_user_ltv,
        src.engaged_sessions_user_ltv,
        src.session_duration_micros_user_ltv
    );
"""
cur.execute(merge)

response = s3.list_objects_v2(Bucket= bucket , Prefix=f'{bucket}/{folder}/')

conn.commit()
conn.close()


Uploaded batch_1.csv to S3.
